<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/wireless-pr3d/ep5g_latency_measurement_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [1]:
import os, re
from getpass import getpass

with open('/content/sdr-test-project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [1]:
!pip uninstall -y moviepy
!pip install jedi
!pip install git+https://github.com/KTH-EXPECA/python-chi

Found existing installation: moviepy 1.0.3
Uninstalling moviepy-1.0.3:
  Successfully uninstalled moviepy-1.0.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.7 MB/s eta 0:00:00
  Cloning https://github.com/KTH-EXPECA/python-chi to /tmp/pip-req-build-2152sdxx
  Running command git clone --filter=blob:none --quiet https://github.com/KTH-EXPECA/python-chi /tmp/pip-req-build-2152sdxx
  Resolved https://github.com/KTH-EXPECA/python-chi to commit ff2e12d0e1219b780b31178ff67eed06cfa39cd9
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/ChameleonCloud/python-blazarclient.git to /tmp/pip-install-ir69hbfu/python-blazarclient_c2567175472b4768967d9aaf38c148a4
  Running command git clone --filter=blob:none --quiet https://github.com/ChameleonCloud/python-blazarclient.git /tmp/pip-install-ir69hbfu/python-blazarclient_c2567175472b4768967d9aaf38c148a4
  Resolved https://github.com/ChameleonCloud/python-blazarclient.git to commit da700b168823d062e3e815cd50048119451e99c

Import packages

In [2]:
import json
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations

In the next cell, we reserve the required equipment and resources to form an end to end experiment setup. We reserve EP5G network, one Advantech router and one worker to run the workloads.

In [4]:
# ep5g reservation
reserve(
    { "type":"network", "name": "ep5g", "net_name": "ep5g-vip", "segment_id": "300", "duration": { "days":7, "hours":0 } }
)

# advantech router reservation
reserve(
    { "type":"network", "name": "adv-02", "net_name": "adv-02", "segment_id": "131", "duration": { "days":7, "hours":0 } }
)

# worker reservation
worker_lease = reserve(
    { "type":"device", "name":"worker-01", "duration": { "days":7, "hours":0 } }
)

worker_reservation_id = worker_lease["reservations"][0]["id"]

leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

2023-07-31 18:19:39.653 | INFO     | chi.expeca:reserve:128 - reserving ep5g
2023-07-31 18:19:43.732 | INFO     | chi.expeca:wait_until_lease_status:34 - waiting 120 seconds for ep5g-lease with id dcb5e671-07b0-4975-9d2b-e708173e0e81 to become "ACTIVE"
2023-07-31 18:19:48.909 | INFO     | chi.expeca:wait_until_lease_status:41 - lease ep5g-lease with id dcb5e671-07b0-4975-9d2b-e708173e0e81 is PENDING.
2023-07-31 18:19:54.062 | INFO     | chi.expeca:wait_until_lease_status:41 - lease ep5g-lease with id dcb5e671-07b0-4975-9d2b-e708173e0e81 is PENDING.
2023-07-31 18:19:59.221 | INFO     | chi.expeca:wait_until_lease_status:41 - lease ep5g-lease with id dcb5e671-07b0-4975-9d2b-e708173e0e81 is PENDING.
2023-07-31 18:20:04.384 | INFO     | chi.expeca:wait_until_lease_status:41 - lease ep5g-lease with id dcb5e671-07b0-4975-9d2b-e708173e0e81 is PENDING.
2023-07-31 18:20:09.548 | INFO     | chi.expeca:wait_until_lease_status:41 - lease ep5g-lease with id dcb5e671-07b0-4975-9d2b-e708173e0e81 is S

TypeError: ignored

In the following section we setup the networking equipment for ep5g (refer to [here](https://kth-expeca.gitbook.io/testbedconfig/enroll/enroll-network-segments/ep5g) for more info).
It contains creation of an edge-net, a router, and some interfaces on the router and routes.

In [6]:
# create edge-net
edgenet = chi.network.create_network("edge-net")
chi.network.create_subnet("edge-net-subnet",edgenet["id"],"10.70.70.0/24",gateway_ip="10.70.70.1",enable_dhcp=False)
logger.success("edge-net is created.")

# create ep5g-vip-router
router = chi.network.create_router("ep5g-vip-router","public")
logger.success("ep5g-vip-router router is created.")
logger.info(f"{json.dumps(router,indent=4)}")

# connect ep5g-vip-net to ep5g-vip-router
ep5gnet = chi.network.get_network("ep5g-vip-net")
portadd = chi.network.add_subnet_to_router(router["id"],ep5gnet["subnets"][0])
logger.success("An interface on ep5g-vip-net is added to the router")

# create edge-net to ep5g-vip-router
edgenet = chi.network.get_network("edge-net")
portadd = chi.network.add_subnet_to_router(router["id"],edgenet["subnets"][0])
logger.success("An interface on edge-net is added to the router")

# add ep5g route to ep5g-vip-router
routeadd = chi.network.add_route_to_router(router["id"],"172.16.0.0/16","10.30.111.10")
logger.success("the route added to the router")

2023-07-31 18:23:21.049 | SUCCESS  | __main__:<cell line: 4>:4 - edge-net is created.


Now the network is ready to run the workloads. We start by runming the edge-node perf-meas container.

In [18]:
edgenet = chi.network.get_network("edge-net")
chi.container.create_container(
    name = "edge-node",
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/mnt/volume/"},
    mounts = [
        {'source': 'edge-volume', 'destination': '/mnt/volume/'}
    ],
    nets = [
        { "network" : edgenet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno12419",
        "networks.1.ip":"10.70.70.3/24",
        "networks.1.routes":"172.16.0.0/16-10.70.70.1",
    },
)
logger.success("created edge-node container.")

2023-07-31 20:03:48.506 | SUCCESS  | __main__:<cell line: 20>:20 - created edge-node container.


Next, we run the end-node perf-meas container.

In [19]:
advnet = chi.network.get_network("adv-02-net")
chi.container.create_container(
    name = "end-node",
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno12429",
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
logger.success("created end-node container.")

2023-07-31 20:04:17.864 | SUCCESS  | __main__:<cell line: 17>:17 - created end-node container.


Run uplink bandwidth test

In [13]:
command = "iperf3 -c 10.70.70.3 -u -b 1G --get-server-output > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="end-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-07-31 18:29:51.139 | INFO     | __main__:<cell line: 7>:7 - {'output': '', 'exit_code': None, 'exec_id': None, 'proxy_url': None}


Run downlink bandwidth test

In [23]:
command = "iperf3 -c 172.16.0.88 -u -b 1G --get-server-output > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2023-07-31 20:11:43.401 | INFO     | __main__:<cell line: 8>:8 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}
